## Amazon Bedrock Introduction

Reference: https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/00_Intro/bedrock_boto3_setup.ipynb

Setting: conda_python3 kernel, ml.c5.xlarge instance

We want the latest version of boto3 and langchain that includes Bedrock, note this may change as the packages update.

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57" \
    "langchain>0.0.306"

In [ ]:
#!pip list

In [ ]:
import boto3

client = boto3.client('bedrock')
runtime = boto3.client('bedrock-runtime')

In [ ]:
model_map = client.list_foundation_models()['modelSummaries']
for model in model_map:
    print(model['modelName'])
    print(model['modelId'])

### Sample Inference Utilizing Claude

In [ ]:
import json

model_id = 'anthropic.claude-v2'
accept = "application/json"
contentType = "application/json"

prompt_data = """Human: Write me a small paragraph saying nice things about me.

Assistant:
"""

body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})

In [ ]:
response = runtime.invoke_model(
    body=body, modelId=model_id, accept=accept, contentType=contentType
)

In [ ]:
response_body = json.loads(response.get("body").read())
print(response_body.get("completion"))

#### Streaming Inference

In [ ]:
from IPython.display import clear_output, display, display_markdown, Markdown

response = runtime.invoke_model_with_response_stream(
    body=body, modelId=model_id, accept=accept, contentType=contentType
)
stream = response.get('body')
output = []

if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['completion']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))

### LangChain Integration

In [ ]:
model_params = {"max_tokens_to_sample": 500,
                "top_k": 100,
                "top_p": .95,
                "temperature": .5}

In [ ]:
from langchain.llms import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Bedrock(
    model_id=model_id,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs=model_params
)

In [ ]:
llm("Give me a recipe for pizza please.")